In [1]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

# Imports

In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [2]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path

In [3]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [4]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/1.2/data_fct_Adrien_point_by_point_reuseinit

matplotlib.rcParams.update({'font.size': 21})

In [5]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [6]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=0)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/RemoteFS/Rivendel/20201006/DC01_RF08/Try00
> Filename : Temp_SimuType2_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG6eV.dat
> myslashpos | [0, 5, 12, 21, 30, 39, 49]
> slashcond | 5
No points deleted because they were not complete
Total number of data directories 588


In [7]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC01_RF08', 'DC01_RF09', 'DC01_RF10', 'DC01_RF11', 'DC03_RF08', 'DC03_RF09', 'DC03_RF10', 'DC03_RF11', 'DC05_RF08', 'DC05_RF09', 'DC05_RF10', 'DC05_RF11', 'DC07_RF08', 'DC07_RF09', 'DC07_RF10', 'DC07_RF11', 'DC09_RF08', 'DC09_RF09', 'DC09_RF10', 'DC09_RF11', 'DC11_RF08', 'DC11_RF09', 'DC11_RF10', 'DC11_RF11', 'DC18_RF08', 'DC18_RF09', 'DC18_RF10', 'DC18_RF11']
000 > DC01_RF08
001 > DC01_RF09
002 > DC01_RF10
003 > DC01_RF11
004 > DC03_RF08
005 > DC03_RF09
006 > DC03_RF10
007 > DC03_RF11
008 > DC05_RF08
009 > DC05_RF09
010 > DC05_RF10
011 > DC05_RF11
012 > DC07_RF08
013 > DC07_RF09
014 > DC07_RF10
015 > DC07_RF11
016 > DC09_RF08
017 > DC09_RF09
018 > DC09_RF10
019 > DC09_RF11
020 > DC11_RF08
021 > DC11_RF09
022 > DC11_RF10
023 > DC11_RF11
024 > DC18_RF08
025 > DC18_RF09
026 > DC18_RF10
027 > DC18_RF11
> condition names ['DC', 'RF']
> number of points 28
> N_ions = 1024
> e_GMol = 50


# all_subdir_reuseinit

In [8]:
## Data loading (1D arrays)
# select data with init to reuse
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu, all_subdir_reuseinit \
= data_retrieve_reuseinit(all_subdir,points_and_coord,condition_parameters,slash_cfg) # stopping_power

data0 = data_name[1]
data2 = data_name[2]
data4 = data_name[3]
Tvar = PMandT[1]

> Points | 28
> Simulations pour chaque point | ['Try00', 'Try01', 'Try02', 'Try03', 'Try04', 'Try05', 'Try06', 'Try07', 'Try08', 'Try09', 'Try10', 'Try11', 'Try12', 'Try13', 'Try14', 'Try15', 'Try16', 'Try17', 'Try18', 'Try19', 'Try20']
> Répertoire : /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try00
> Filename : Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [0, 5, 12, 21, 30, 39, 49]
> slashcond | 5
No points deleted because they were not complete
Total number of data directories 2800
Hello
Point n° 0
00 - 00  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try00/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 01  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try01/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 02  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try02/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 03  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try03/Temp_Sim

02 - 19  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try19/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 20  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try20/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
Point n° 3
03 - 00  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF11/Try00/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.1000D+01V_D1.0_S1.0RFG
03 - 01  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF11/Try01/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.1000D+01V_D1.0_S1.0RFG
03 - 02  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF11/Try02/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.1000D+01V_D1.0_S1.0RFG
03 - 03  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF11/Try03/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.1000D+01V_D1.0_S1.0RFG
03 - 04  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF11/Try04/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.1000D+01V_D1.0_S1.0RFG
03 - 05  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF11/Try05/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.1000

Point n° 6
06 - 00  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try00/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 01  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try01/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 02  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try02/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 03  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try03/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 04  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try04/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 05  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try05/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 06  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try06/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 07  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try07/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000

09 - 02  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF09/Try02/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
09 - 03  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF09/Try03/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
09 - 04  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF09/Try04/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
09 - 05  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF09/Try05/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
09 - 06  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF09/Try06/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
09 - 07  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF09/Try07/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
09 - 08  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF09/Try08/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
09 - 09  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF09/Try09/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.3000D+01V_D1.0_

12 - 04  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF08/Try04/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.4000D+01V_D1.0_S1.0RFG
12 - 05  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF08/Try05/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.4000D+01V_D1.0_S1.0RFG
12 - 06  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF08/Try06/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.4000D+01V_D1.0_S1.0RFG
12 - 07  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF08/Try07/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.4000D+01V_D1.0_S1.0RFG
12 - 08  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF08/Try08/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.4000D+01V_D1.0_S1.0RFG
12 - 09  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF08/Try09/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.4000D+01V_D1.0_S1.0RFG
12 - 10  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF08/Try10/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.4000D+01V_D1.0_S1.0RFG
12 - 11  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF08/Try11/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.4000D+01V_D1.0_

15 - 06  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF11/Try06/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG
15 - 07  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF11/Try07/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG
15 - 08  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF11/Try08/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG
15 - 09  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF11/Try09/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG
15 - 10  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF11/Try10/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG
15 - 11  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF11/Try11/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG
15 - 12  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF11/Try12/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.4000D+01V_D1.0_S1.0RFG
15 - 13  >  /home/adrian/RemoteFS/Rivendel/20200903/DC07_RF11/Try13/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.4000D+01V_D1.0_

18 - 08  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try08/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
18 - 09  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try09/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
18 - 10  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try10/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
18 - 11  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try11/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
18 - 12  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try12/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
18 - 13  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try13/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
18 - 14  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try14/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
18 - 15  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try15/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_

21 - 10  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try10/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.7000D+01V_D1.0_S1.0RFG
21 - 11  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try11/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.7000D+01V_D1.0_S1.0RFG
21 - 12  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try12/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.7000D+01V_D1.0_S1.0RFG
21 - 13  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try13/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.7000D+01V_D1.0_S1.0RFG
21 - 14  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try14/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.7000D+01V_D1.0_S1.0RFG
21 - 15  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try15/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.7000D+01V_D1.0_S1.0RFG
21 - 16  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try16/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.7000D+01V_D1.0_S1.0RFG
21 - 17  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try17/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.7000D+01V_D1.0_

24 - 12  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try12/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 13  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try13/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 14  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try14/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 15  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try15/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 16  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try16/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 17  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try17/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 18  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try18/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
24 - 19  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF08/Try19/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.6000D+01V_D1.0_

27 - 14  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try14/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 15  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try15/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 16  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try16/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 17  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try17/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 18  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try18/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 19  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try19/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 20  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try20/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
Time elapsed:  566.17806 s
Time elapsed:  9.436300999999998 m


In [9]:
# Automatic retrieve of GMol energy
aux_energie = file_path.find('0RFG')
aux_energie_unit = file_path.find('eV')
E0_GMol = file_path[aux_energie+4:aux_energie_unit+2]
E0_GMol = re.sub('k','000',E0_GMol)
E0_GMol = int(E0_GMol.strip('eV'))
print(E0_GMol)

6


In [10]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

1.0


In [118]:
file_cfg_reuseinit, slash_cfg_reuseinit, all_subdir_reuseinit = load_gui('/home/adrian')
all_subdir_reuseinit_aux = []
for k,l in enumerate(all_subdir_reuseinit):
    for number in range(0,len(num_runs)):
        if int(l[-2:]) == number:
            all_subdir_reuseinit_aux.append(l)

> Répertoire : /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try00
> Filename : Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [0, 5, 12, 21, 30, 39, 49]
> slashcond | 5
No points deleted because they were not complete
Total number of data directories 2800


In [146]:
k = 25*13 + 18
k = 30

address = all_subdir[k]
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
print(onlyfiles)
print('{}/{}'.format(address,sort(onlyfiles)[1].strip('.dat')),'\n')

address_reuseinit = all_subdir_reuseinit_aux[k]
onlyfiles_reuseinit = [f for f in listdir(address_reuseinit) if isfile(join(address_reuseinit, f)) and not "xva" in f and ".dat" in f]
print(onlyfiles_reuseinit)
print('{}/{}'.format(address_reuseinit,sort(onlyfiles_reuseinit)[2].strip('.dat')))

['Temp_SimuType4_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG1keV.dat', 'Temp_SimuType2_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG1keV.dat']
/home/adrian/RemoteFS/Rivendel/20200929/DC01_RF08/Try01/Temp_SimuType4_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG1keV 

['Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG.dat', 'Temp_SimuType4_01_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG50eV.dat', 'Temp_SimuType2_01_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG50eV.dat']
/home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try00/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG


# Plots
Nuage d'ions, Temp et Pos

## Load temperature from Langevin files

In [513]:
## Data loading with hardcode names
eta = 1.5e-20

plot_point = 12
plot_try = 5
# str_eta_chiffre    = float(f'{eta[plot_point]:1.2e}'[:4])/10
# str_eta_puissance  = int(re.sub('e','D',f'{eta[plot_point]:1.2e}'[6:]))-1
# str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
str_eta = '0.15D-19'
dirname_hc = str(data0[plot_point][plot_try][:-58])+'/'
# Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
data0_hc    = '/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data2_hc    = '/Temp_SimuType2_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'

print(dirname_hc)
print(datainit_hc)

N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T =\
load_Temp_init_bin_Lan(dirname_hc+datainit_hc,1)

/home/adrian/RemoteFS/Rivendel/20200903/DC07_RF08/Try05/
Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5
len save_T [55440]
[4] [4] [8] [8] [8] [55440]
N_ions 1024
j_save 990
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 4.25522243e-05 2.81376418e-03 1.81936160e-07
 4.68361626e+01 1.34627280e+00 6.57232457e+01]


In [514]:
# Get T and fluo PM  All SimuType
init_data = data0[plot_point][plot_try][:myslashpos[slashcond]+16]
evol_data = data2[plot_point][plot_try][:myslashpos[slashcond]+16]
inje_data = data4[plot_point][plot_try][:myslashpos[slashcond]+16]
# print(init_data,evol_data,inje_data)
# onlyfiles = [f for f in listdir(evol_data) if isfile(join(evol_data, f)) and "SimuType" in f and ".dat" in f]
# print(sort(onlyfiles))
# onlyfiles_reuseinit = [f for f in listdir(init_data) if isfile(join(init_data, f)) and "SimuType" in f and ".dat" in f]
# print(sort(onlyfiles_reuseinit))

ta, te, ti = plot_T_and_PM_InitQ_Inje_Evol(init_data,evol_data,inje_data,
                                           flag_plot=1,fig_name='0o0o0o0',xlim1=(0,11),ylim1=(2e-4,500),ylim2=(2,65))

# tt2, T_CM2, T_aux2, PM2
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

In [154]:
# recover duration for each step of simu (i_free__fly, i_laser_fly ...)

address = all_subdir_reuseinit[plot_point]
print(address)
info_files = sort( [f for f in listdir(address) if isfile(join(address, f)) and ".info" in f] )
print(sort(info_files))


tmp_info = loadtxt(data0[plot_point][0][:-58]+'/'+info_files[0],encoding ='utf-8',
            skiprows=1, usecols=(0),
            unpack=True)
eta = tmp_info[-2]
i_free__fly_Lan = tmp_info[-1]

tmp_info = loadtxt(data0[0][0][:-58]+'/'+'xva_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.info',encoding ='utf-8',
            skiprows=0, usecols=(0),
            unpack=True)
i_laser_fly_quench = tmp_info[-2]
i_laser_fly = tmp_info[-1]

n_dt = 100
dt = 1/2e6

print(eta)
print(i_free__fly_Lan)
print(i_laser_fly_quench)
print(i_laser_fly)

/home/adrian/RemoteFS/Rivendel/20200903/DC01_RF09/Try02
['Langevin_cooling.info'
 'xva_SimuType2_01_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG50eV.info'
 'xva_SimuType4_01_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG50eV.info'
 'xva_SimuTypeQ_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG.info']
1.5e-20
1000.0
8000.0
6000.0


In [516]:
# Plot temperature Langevin + md laser  ALL SimuType

tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
tt_md = ta[0]
T_aux = te[0]

tt = concatenate((tt_Lan,tt_md))
T_all = concatenate((save_T[:,4:],T_aux),axis=0)

PM = ti[0]

#fig_name = file_name[-9:] 
figure('Temperature'); clf()
ax1 = subplot(211)
ax1.semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
ax1.semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
ax1.semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#semilogy(tt_Lan*1.e3,save_T[:,0], label='Tx',color='C0')
#semilogy(tt_Lan*1.e3,save_T[:,1], label='Ty',color='C1')
#semilogy(tt_Lan*1.e3,save_T[:,2], label='Tz',color='C2')
ax1.vlines(ta[1]*1e3,1e-4,1e4,color='red')
ax1.vlines(ta[2]*1e3,1e-4,1e4,color='red')
time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='green')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
# annotate('Laser ON', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul')
,
# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

ax2 = subplot(212,sharex=ax1)
plot(tt_md*1e3,PM, label='Tx')
time_switch = i_free__fly_Lan*dt
ax2.vlines(time_switch*1e3,1e-4,90,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax2.vlines(time_switch*1e3,1e-4,90,color='green')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax2.vlines(time_switch*1e3,1e-4,90,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,90,color='purple')
ax2.grid()
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time[ms]')
ylabel('Counts')

plt.tight_layout()
subplots_adjust(hspace=0.015)

In [67]:
# Plot all temp for all conditions
fname = 'Temp_allpoints'
cm = pylab.get_cmap('rainbow')

fig, ax = plt.subplots(4, 7,num=fname,clear=True,sharex=True,sharey=True)  # (a,b)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

to_plot = 0

for l in range(0,7): # b
    for k in range(0,4): # a
        plot_point = to_plot
#             str_eta_chiffre    = float(f'{eta[patate]:1.2e}'[:4])/10
#             str_eta_puissance  = int(re.sub('e','D',f'{eta[patate]:1.2e}'[6:]))-1
#             str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
        str_eta = '0.15D-19'
        
        for plot_try in range(3):
            info_files = sort( [f for f in listdir(data0[plot_point][plot_try][:-58]) if isfile(join(data0[plot_point][plot_try][:-58], f)) and ".bin" and "3D" in f] )
            N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
            load_Temp_init_bin_Lan(data0[plot_point][plot_try][:-57]+info_files[0].strip('.bin'),flag_print=0)
            
            init_data = data0[plot_point][plot_try][:myslashpos[slashcond]+16]
            evol_data = data2[plot_point][plot_try][:myslashpos[slashcond]+16]
            inje_data = data4[plot_point][plot_try][:myslashpos[slashcond]+16]
            
            # Get T and fluo PM
            ta, te, ti = plot_T_and_PM_InitQ_Inje_Evol(init_data,evol_data,inje_data,
                                           flag_plot=0,fig_name='0o0o0o0',xlim1=(0,11),ylim1=(2e-4,500),ylim2=(2,65))

            tt_Lan = linspace(0,dt_j_save_next*j_save,j_save)
            tt_md = ta[0]
            T_aux = te[0]

    #         tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
    #         tt_md = ta[0]
    #         T_aux = te[0]
    #         tt = concatenate((tt_Lan,tt_md))
    #         T_all = concatenate((save_T[:,4:],T_aux),axis=0)

            tt = concatenate((tt_Lan,tt_md))
            T_all = concatenate((save_T[:,4:],T_aux),axis=0)

#             ax[3-k,l].semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
#             ax[3-k,l].semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
            ax[3-k,l].semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#         ax[k,l].vlines(ta[1]*1e3,1e-4,1e4,color='red')
#         ax[k,l].vlines(ta[2]*1e3,1e-4,1e4,color='red')
# print(i_free__fly_Lan)
# print(i_laser_fly_quench)
# print(i_laser_fly)
        time_switch = i_free__fly_Lan*dt
        ax[3-k,l].vlines(time_switch*1e3,1e-4,1e4,color='blue')
        time_switch = (i_free__fly_Lan+i_laser_fly)*dt
        ax[3-k,l].vlines(time_switch*1e3,1e-4,1e4,color='green')
        time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
        ax[3-k,l].vlines(time_switch*1e3,1e-4,1e4,color='orange')
        time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
        ax[3-k,l].vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
        ax[3-k,l].set_yticks([1e-4,1e-2,1,1e2,1e4])
        ax[3-k,l].grid()
        ax[3-k,l].set_xlim(7.49,7.57)
        im = ax[3-k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.04))
        ax[3-k,l].annotate(str(list(dict.fromkeys(points_and_coord))[plot_point]), xy=(0.20, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
        size=12, ha='left', va='top',
        bbox=dict(boxstyle='round', fc=cm(detect_rate[to_plot])))
        to_plot += 1
        if to_plot == 28:
            break
    if to_plot == 28:
        break
# ax[3].set_xlim(-10,4000)
# ax[k,l].set_ylim(2e-4,1e4)
fig.suptitle(str(eta) + 'Kg.s-1')
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig(fname+'.png') 

len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 2.01746889e-06 9.62312372e-05 1.83416321e-09
 4.37808518e+01 1.20427139e+00 7.00431976e+01]
len save_T [55440]
[5.05000000e-07 1.49328772e-04 5.10237332e-04 1.01786500e-06
 4.39755749e+01 1.27465605e+00 7.06015073e+01]


NameError: name 'i_free__fly_Lan' is not defined

In [232]:
# Plot all fluo for all conditions
fname = 'Fluo_allpoints'
cm = pylab.get_cmap('rainbow')

fig, ax = plt.subplots(4, 7,num=fname,clear=True,sharex=True,sharey=True)  # (a,b)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

to_plot = 0

for l in range(0,7): # b
    for k in range(0,4): # a
        plot_point = to_plot
#             str_eta_chiffre    = float(f'{eta[patate]:1.2e}'[:4])/10
#             str_eta_puissance  = int(re.sub('e','D',f'{eta[patate]:1.2e}'[6:]))-1
#             str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
        str_eta = '0.15D-19'
    
        for plot_try in range(25):
            dirname_hc = str(data0[plot_point][plot_try])+'/'
            # Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
            datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
            N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
            load_Temp_init_bin_Lan(dirname_hc+datainit_hc,flag_print=0)
            # Get T and fluo PM
            onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and 'SimuType' in f and '.dat' in f]
            ta, te, ti = plot_T_and_PM_InitQ_Inje_Evol(dirname_hc,sort(onlyfiles)[2].strip('.dat')[4:],flag_plot = 0,fig_name = 'Temp_allpoints')

            tt_Lan = linspace(0,dt_j_save_next*j_save,j_save)
            tt_md = ta[0]
            T_aux = ti[0]
        
#         tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
#         tt_md = ta[0]
#         T_aux = te[0]
#         tt = concatenate((tt_Lan,tt_md))
#         T_all = concatenate((save_T[:,4:],T_aux),axis=0)

            tt = tt_md
            T_all = T_aux

            ax[3-k,l].plot(tt*1e3,T_aux, label='Fluo',color='C0')
#         ax[3-k,l].semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
#         ax[3-k,l].semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#         ax[k,l].vlines(ta[1]*1e3,1e-4,1e4,color='red')
#         ax[k,l].vlines(ta[2]*1e3,1e-4,1e4,color='red')
# print(i_free__fly_Lan)
# print(i_laser_fly_quench)
# print(i_laser_fly)
        time_switch = i_free__fly_Lan*dt
        ax[3-k,l].vlines(time_switch*1e3,1e-4,1e4,color='blue')
        time_switch = (i_free__fly_Lan+i_laser_fly)*dt
        ax[3-k,l].vlines(time_switch*1e3,1e-4,1e4,color='green')
        time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
        ax[3-k,l].vlines(time_switch*1e3,1e-4,1e4,color='orange')
        time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
        ax[3-k,l].vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
        ax[3-k,l].grid()
        im = ax[3-k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.2))
        ax[3-k,l].annotate(str(list(dict.fromkeys(points_and_coord))[plot_point]), xy=(0.20, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
        size=12, ha='left', va='top',
        bbox=dict(boxstyle='round', fc=cm(detect_rate[to_plot])))
        to_plot += 1
        if to_plot == 28:
            break
    if to_plot == 28:
        break
# ax[3].set_xlim(-10,4000)
ax[3-k,l].set_xlim(7.24,7.80)
ax[3-k,l].set_ylim(0,100)
fig.suptitle(str(eta) + 'Kg.s-1')
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig(fname+'.png')

len save_T [55440]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]
len save_T [55440]
[5.05000000e-07 2.01746889e-06 9.62312372e-05 1.83416321e-09
 4.37808518e+01 1.20427139e+00 7.00431976e+01]
len save_T [55440]
[5.05000000e-07 1.49328772e-04 5.10237332e-04 1.01786500e-06
 4.39755749e+01 1.27465605e+00 7.06015073e+01]
len save_T [55440]
[5.05000000e-07 1.31772544e-05 3.57273936e-03 2.91283383e-07
 4.30875300e+01 1.06822833e+00 6.92841434e+01]
len save_T [55440]
[5.05000000e-07 1.03570089e-06 1.24300504e-05 3.09298137e-06
 4.29291943e+01 1.22294703e+00 6.88519033e+01]
len save_T [55440]
[5.05000000e-07 2.51524433e-06 3.91204513e-03 6.05211860e-06
 4.33997404e+01 1.19106214e+00 6.88822912e+01]
len save_T [55440]
[5.05000000e-07 4.26082011e-06 1.28150563e-03 1.91081500e-06
 4.41452320e+01 1.40883631e+00 7.06705779e+01]
len save_T [55440]
[5.05000000e-07 1.48921171e-05 3.91110107e-03 7.01723638e-07
 4.48908701e+01 3.00159364e+00 

len save_T [55440]
[5.05000000e-07 2.37011413e-05 3.00517750e-03 3.54148810e-07
 4.39449007e+01 1.74550329e+00 7.16624837e+01]
len save_T [55440]
[5.05000000e-07 1.09011141e-06 1.14544331e-03 4.91732881e-08
 4.35115072e+01 1.16400347e+00 6.94604770e+01]
len save_T [55440]
[5.05000000e-07 3.73307795e-05 1.04959353e-04 1.45043243e-07
 4.46603422e+01 1.13501034e+00 7.20904300e+01]
len save_T [55440]
[5.05000000e-07 7.08753446e-06 3.59547023e-03 4.11223309e-06
 4.23579027e+01 1.16719087e+00 6.84593840e+01]
len save_T [55440]
[5.05000000e-07 6.57620827e-06 1.91365289e-03 1.01102349e-05
 4.46800139e+01 1.38978151e+00 7.16475108e+01]
len save_T [55440]
[5.05000000e-07 9.68552535e-05 1.36342298e-02 1.93433671e-08
 4.56093707e+01 1.35470121e+00 7.34939986e+01]
len save_T [55440]
[5.05000000e-07 2.08840559e-06 8.06209935e-03 1.09143267e-05
 4.41500218e+01 1.22457914e+00 7.10570695e+01]
len save_T [55440]
[5.05000000e-07 4.33923580e-06 2.13136639e-02 8.05734851e-07
 4.35564646e+01 1.39640542e+00 

len save_T [55440]
[5.05000000e-07 1.62360095e-05 1.38118103e-03 4.94255119e-05
 4.71246705e+01 1.60896282e+00 7.25989780e+01]
len save_T [55440]
[5.05000000e-07 3.26162906e-05 8.17974337e-03 5.56663465e-06
 4.65330259e+01 1.51042011e+00 7.17692918e+01]
len save_T [55440]
[5.05000000e-07 6.19681956e-05 4.16643436e-04 5.85294395e-05
 4.42192375e+01 1.27436424e+00 6.80953210e+01]
len save_T [55440]
[5.05000000e-07 1.51331745e-05 8.16814279e-04 3.19104698e-05
 4.48726063e+01 1.38872794e+00 6.92039381e+01]
len save_T [55440]
[5.05000000e-07 5.73961721e-06 1.12137460e-06 4.98881528e-06
 4.57412496e+01 1.17192651e+00 6.97970549e+01]
len save_T [55440]
[5.05000000e-07 7.53106402e-05 5.85280591e-04 2.27207084e-05
 4.51900534e+01 1.32382520e+00 6.99778688e+01]
len save_T [55440]
[5.05000000e-07 1.81023902e-04 1.37556200e-04 2.21986648e-05
 4.44994467e+01 1.16277986e+00 6.82346226e+01]
len save_T [55440]
[5.05000000e-07 1.14806672e-05 7.03415116e-04 4.68514343e-07
 4.46768199e+01 1.39877665e+00 

len save_T [55440]
[5.05000000e-07 9.85796390e-06 2.33020388e-04 1.84321858e-06
 4.67738785e+01 1.36757240e+00 7.22506901e+01]
len save_T [55440]
[5.05000000e-07 1.45778818e-04 1.98932250e-03 1.28151397e-06
 4.44890517e+01 1.18743521e+00 6.82644011e+01]
len save_T [55440]
[5.05000000e-07 3.44405909e-05 1.19125388e-03 2.63565892e-05
 4.49834025e+01 1.55439768e+00 6.93458737e+01]
len save_T [55440]
[5.05000000e-07 2.80718729e-06 2.30420595e-04 1.05114681e-05
 4.52901921e+01 1.24568917e+00 6.93628627e+01]
len save_T [55440]
[5.05000000e-07 1.40915019e-05 1.19512132e-02 1.74970073e-07
 4.57375531e+01 1.44472293e+00 6.99291525e+01]
len save_T [55440]
[5.05000000e-07 1.76865858e-05 2.79998056e-03 4.23705269e-05
 4.64372065e+01 1.39355749e+00 6.82470294e+01]
len save_T [55440]
[5.05000000e-07 6.12851692e-05 5.64386325e-03 2.53788788e-06
 4.44980860e+01 1.14127418e+00 6.63332880e+01]
len save_T [55440]
[5.05000000e-07 1.25684118e-05 4.36083225e-05 4.69045945e-06
 4.73961608e+01 1.73460782e+00 

len save_T [55440]
[5.05000000e-07 2.09091086e-06 2.77725540e-05 1.22912345e-05
 4.76562561e+01 1.39073101e+00 6.94939829e+01]
len save_T [55440]
[5.05000000e-07 3.96839793e-05 2.23107905e-04 1.60860525e-05
 4.71969161e+01 1.45923656e+00 6.96803477e+01]
len save_T [55440]
[5.05000000e-07 6.26577857e-06 8.90741433e-04 4.51559237e-06
 4.59449445e+01 1.61679209e+00 6.68412353e+01]
len save_T [55440]
[5.05000000e-07 7.53186210e-05 5.56125073e-04 3.66239998e-05
 4.67035541e+01 1.30802634e+00 7.34280690e+01]
len save_T [55440]
[5.05000000e-07 3.25186682e-08 8.17907237e-03 2.02401763e-05
 4.59108495e+01 1.38802690e+00 6.69895374e+01]
len save_T [55440]
[5.05000000e-07 6.73043674e-06 1.68849396e-03 6.55165017e-05
 4.36014361e+01 1.23615677e+00 6.35337219e+01]
len save_T [55440]
[5.05000000e-07 7.37838190e-05 2.13985386e-03 9.57105447e-07
 4.59418291e+01 1.24746689e+00 6.71017214e+01]
len save_T [55440]
[5.05000000e-07 1.81092364e-05 1.54913954e-02 5.77981357e-05
 4.43897593e+01 1.26515246e+00 

...

len save_T [55440]
[5.05000000e-07 1.30332906e-04 9.64768144e-06 1.18355837e-04
 4.83315693e+01 1.57823681e+00 6.84028673e+01]
len save_T [55440]
[5.05000000e-07 6.72935448e-05 1.02339759e-03 9.57152674e-05
 4.63342731e+01 1.59679951e+00 6.51315574e+01]
len save_T [55440]
[5.05000000e-07 1.27284847e-04 3.30810243e-03 6.10718889e-05
 4.72446756e+01 1.63384676e+00 6.62481453e+01]
len save_T [55440]
[5.05000000e-07 1.79015801e-05 7.84549093e-04 2.95253565e-06
 4.55534869e+01 1.51157004e+00 6.37324484e+01]
len save_T [55440]
[5.05000000e-07 3.94956173e-05 5.02033722e-03 7.82337192e-05
 4.55791421e+01 1.42776878e+00 6.35088925e+01]
len save_T [55440]
[5.05000000e-07 2.19689569e-05 1.04706557e-02 4.90562970e-05
 4.78406968e+01 1.33725913e+00 6.63024150e+01]
len save_T [55440]
[5.05000000e-07 3.53481287e-06 8.98651800e-03 3.75052693e-05
 4.85014384e+01 1.69266895e+00 6.67894354e+01]
len save_T [55440]
[5.05000000e-07 4.14285611e-06 2.32507342e-03 3.99122133e-05
 4.61827946e+01 1.29188882e+00 

len save_T [55440]
[5.05000000e-07 2.14491124e-05 9.71181175e-04 6.43577041e-05
 4.66846520e+01 1.45014230e+00 6.15658842e+01]
len save_T [55440]
[5.05000000e-07 1.22241214e-06 5.68681675e-03 1.28634002e-05
 4.76737991e+01 1.50714916e+00 6.29091812e+01]
len save_T [55440]
[5.05000000e-07 1.95846501e-07 1.51090767e-03 1.23982966e-07
 4.69343613e+01 1.20258681e+00 6.23117052e+01]
len save_T [55440]
[5.05000000e-07 1.38721733e-06 8.25599467e-04 2.63925718e-05
 4.99448855e+01 1.63470910e+00 6.62601369e+01]
len save_T [55440]
[5.05000000e-07 1.75244444e-06 3.00450933e-03 1.60319298e-04
 4.83248765e+01 1.62679938e+00 6.47286255e+01]
len save_T [55440]
[5.05000000e-07 4.55158440e-06 5.53107218e-03 1.33208693e-05
 4.66132105e+01 1.26696357e+00 6.22763776e+01]
len save_T [55440]
[5.05000000e-07 5.56526718e-06 6.90010734e-05 1.75668917e-04
 4.65100286e+01 1.42768137e+00 6.14504125e+01]
len save_T [55440]
[5.05000000e-07 7.30083090e-05 3.15218984e-04 4.23862174e-06
 4.94088353e+01 1.79898600e+00 

len save_T [55440]
[5.05000000e-07 2.82212032e-05 6.93342374e-04 3.51028423e-07
 4.58989821e+01 1.17570233e+00 5.49465081e+01]
len save_T [55440]
[5.05000000e-07 1.63501930e-06 2.04991953e-03 2.36866081e-06
 4.92251147e+01 1.57467416e+00 5.86355004e+01]
len save_T [55440]
[5.05000000e-07 4.01380809e-06 1.14037413e-03 1.09870615e-04
 4.96442198e+01 1.74818480e+00 6.03636083e+01]
len save_T [55440]
[5.05000000e-07 2.46800239e-06 6.50222314e-03 6.88332677e-05
 4.87262594e+01 1.40755885e+00 5.86460053e+01]
len save_T [55440]
[5.05000000e-07 6.09779076e-06 7.56728483e-05 6.07362331e-05
 4.89827702e+01 1.38632345e+00 5.91673089e+01]
len save_T [55440]
[5.05000000e-07 3.99433963e-07 8.69783790e-05 1.20840381e-04
 4.82775901e+01 1.63689422e+00 5.81073001e+01]
len save_T [55440]
[5.05000000e-07 2.56775369e-06 1.52641556e-03 4.38692001e-04
 5.08385315e+01 1.69253794e+00 6.12907621e+01]
len save_T [55440]
[5.05000000e-07 4.82391474e-09 1.22427700e-03 2.24651634e-05
 5.03104988e+01 1.59392177e+00 

len save_T [55440]
[5.05000000e-07 2.29958672e-05 1.50552780e-04 7.01548969e-05
 4.78196090e+01 1.66017377e+00 5.79717854e+01]
len save_T [55440]
[5.05000000e-07 4.88704795e-06 3.09779202e-02 3.03262343e-04
 4.72629127e+01 1.32354106e+00 5.66695242e+01]
len save_T [55440]
[5.05000000e-07 1.06202456e-05 2.91034460e-04 3.03729836e-05
 4.85050336e+01 1.47963216e+00 5.85502326e+01]
len save_T [55440]
[5.05000000e-07 9.17681182e-07 1.15229836e-02 1.68621926e-04
 5.00898974e+01 1.84863736e+00 5.98771676e+01]
len save_T [55440]
[5.05000000e-07 7.17927332e-07 7.13300577e-04 1.31319080e-05
 4.81656031e+01 1.25959661e+00 5.86590762e+01]
len save_T [55440]
[5.05000000e-07 9.07603537e-06 2.99331850e-04 4.83099949e-07
 4.89668587e+01 1.41942009e+00 5.89104284e+01]
len save_T [55440]
[5.05000000e-07 3.13038592e-05 1.05790274e-03 5.12513656e-04
 5.07933036e+01 1.57350426e+00 5.86737419e+01]
len save_T [55440]
[5.05000000e-07 4.07380033e-06 4.08112789e-04 1.07487617e-05
 4.82540300e+01 1.36464139e+00 

len save_T [55440]
[5.05000000e-07 1.45214527e-08 1.93489046e-03 1.28363477e-04
 4.83210037e+01 1.37186983e+00 6.15892420e+01]
len save_T [55440]
[5.05000000e-07 8.54860055e-07 7.99609912e-03 3.13278004e-05
 4.78038045e+01 1.31630441e+00 6.05019671e+01]
len save_T [55440]
[5.05000000e-07 8.85226661e-06 1.16493866e-03 1.06195364e-04
 5.00623258e+01 1.92150869e+00 6.24391137e+01]
len save_T [55440]
[5.05000000e-07 5.69647151e-06 6.81908350e-04 2.97847834e-04
 4.78386947e+01 1.38123776e+00 6.01849425e+01]
len save_T [55440]
[5.05000000e-07 1.73771109e-05 1.07834275e-03 6.84793046e-05
 4.69514496e+01 1.36072044e+00 5.87168181e+01]
len save_T [55440]
[5.05000000e-07 3.32184523e-05 8.88183265e-05 3.22953288e-04
 4.92761566e+01 1.68405525e+00 6.20654133e+01]
len save_T [55440]
[5.05000000e-07 5.66987203e-06 1.26050607e-03 8.00406941e-05
 4.77475901e+01 1.52548615e+00 6.01578260e+01]
len save_T [55440]
[5.05000000e-07 1.58665046e-05 6.18120863e-04 2.18397236e-04
 4.60043801e+01 1.17354124e+00 

In [20]:
Gmol_data[2]

mean_std_deltaEc = []
detect_rate = []
detect_cnt = 0
detect_threshold = 1
renorm = 0.5*m_GM/C_e

# mean_std_deltaEc = [mean(Gmol_data[0])*renorm,std(Gmol_data[0])*renorm,mean(detec_delta_Ec)*renorm,std(detec_delta_Ec)*renorm]

for k in range(shape(Gmol_data[2])[0]):
    detec_delta_Ec = []
    print('k',k)
    detect_rate.append(0)
    for l,m in enumerate(Gmol_data[2][k]):
        print('... l',l)
        if m > detect_threshold:
            detect_cnt += 1
            detect_rate[k] += 1
            detec_delta_Ec.append(Gmol_data[0][k][l])
    detect_rate[k] = detect_rate[k] / shape(Gmol_data[2])[1]
    mean_std_deltaEc.append(list(multiply([mean(Gmol_data[0][k]),std(Gmol_data[0][k]),mean(detec_delta_Ec),std(detec_delta_Ec)],renorm)))
    
for n,o in enumerate(list(dict.fromkeys(points_and_coord))):
    print(o)
    print(detect_rate[n])

k 0
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... l 12
... l 13
... l 14
... l 15
... l 16
... l 17
... l 18
... l 19
... l 20
k 1
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... l 12
... l 13
... l 14
... l 15
... l 16
... l 17
... l 18
... l 19
... l 20
k 2
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... l 12
... l 13
... l 14
... l 15
... l 16
... l 17
... l 18
... l 19
... l 20
k 3
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... l 12
... l 13
... l 14
... l 15
... l 16
... l 17
... l 18
... l 19
... l 20
k 4
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... l 12
... l 13
... l 14
... l 15
... l 16
... l 17
... l 18
... l 19
... l 20
k 5
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
.

/home/adrian/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/adrian/.local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/adrian/.local/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/adrian/.local/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/adrian/.local/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [21]:
Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]

my_order = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,24,25,26,27,20,21,22,23]

In [22]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [247]:
udc_var = linspace(0.001,8.2,1e3)
urf_low_lim  = sqrt( udc_var * kappa_simion*Omega**2*m_Ca*r0**4/ (2*C_e*d0**2) )
urf_high_lim = m_Ca*r0**2*Omega**2/4/C_e*0.9

my_omegaz_2 = 2*kappa_simion*C_e*udc_var/m_Ca/d0**2
my_omegap_2 = 0.5 * 2*kappa_simion*C_e*udc_var/m_Ca/d0**2
my_omegar_2 = my_omegap_2 - 0.5

/home/adrian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  """Entry point for launching an IPython kernel.


In [253]:
x = []
y = []
z = []

for g,h in enumerate(points_and_coord):
    my_udc = Udc[int(points_and_coord[h][0])]
    my_urf = Urf[int(points_and_coord[h][1])]
    x.append(my_udc)
    y.append(my_urf)
    z.append(detect_rate[g])
    

    
fname = '2D_GMol_var_fluo'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=400,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.set_xlabel(r'$U_{DC}$ [V]')
ax.set_ylabel(r'$U_{RF}$ [V]')
ax.grid()

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=13,
                ha='center',va='center',weight='bold',zorder = 1000)
    ax.annotate(str(txt), (x[i], y[i]+4e4),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

In [254]:
x = []
y = []
z = []

for g,h in enumerate(points_and_coord):
    my_udc = Udc[int(points_and_coord[h][0])]
    my_urf = Urf[int(points_and_coord[h][1])]
    x.append(my_udc)
    y.append(my_urf)
    z.append(detect_rate[g])

    
fname = '2D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=400,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.plot(udc_var,urf_low_lim)
ax.plot(udc_var,[urf_high_lim]*len(udc_var))
ax.set_xlabel(r'$U_{DC}$ [V]')
ax.set_ylabel(r'$U_{RF}$ [V]')
ax.grid()

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=13,
                ha='center',va='center',weight='bold',zorder = 1000)
    ax.annotate(str(txt), (x[i], y[i]+4e4),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

In [255]:
x = []
y = []
z = []
omegaz_2 = zeros(len(list(points_and_coord)))
omegar_2 = zeros(len(list(points_and_coord)))
omegap_2 = zeros(len(list(points_and_coord)))
for g,h in enumerate(points_and_coord):
    my_udc = Udc[int(points_and_coord[h][0])]
    my_urf = Urf[int(points_and_coord[h][1])]
#     x.append(my_udc)
#     y.append(my_urf)
    z.append(detect_rate[g])
    
    omegaz_2[g] = 2*kappa_simion*C_e/m_Ca/d0**2*my_udc
    omegap_2[g] = 2*C_e**2/m_Ca**2/r0**4/Omega**2*my_urf**2
    omegar_2[g] = omegap_2[g] - 0.5*omegaz_2[g]
    x.append(sqrt(omegaz_2[g]))
    y.append(sqrt(omegar_2[g]))
    
fname = '2D_GMol_var_fluo_contour_wzwr'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=400,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.set_xlabel(r'$\omega_{z}$ [Hz]')
ax.set_ylabel(r'$\omega_{r}$ [Hz]')
ax.grid()

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=13,
                ha='center',va='center',weight='bold',zorder = 1000)
    ax.annotate(str(txt), (x[i], y[i]+4e4),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

In [256]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(mean(dim_nu[g,:,0])/mean(dim_nu[g,:,2]))
    y.append(mean(dim_nu[g,:,0])*1000)
    z.append(detect_rate[g])
    
    
fname = '2D_GMol_var_fluo_contour_Rvs_RoverL'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=250,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.set_xlabel(r'mean R/L')
ax.set_ylabel(r'mean R [µm]')

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=10,
                ha='center',va='center',weight='bold',zorder = 1000)
    # ax.annotate(str(txt), (x[i], y[i]-0.70),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.05))
ax.minorticks_on()
# ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.grid(which='major', linestyle='-', linewidth='1', color='k')
ax.grid(b=True, which='minor', color='k', linestyle=':')
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

In [257]:
address = all_subdir[14*31]
# dirname_hc = str(data0[1][0])+'/'
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and 'SimuType' in f and '.dat' in f]
my_file = '{}/xva{}'.format(address,sort(onlyfiles)[2].strip('.dat')[4:])
plot_XYZ(my_file,fig_name='2',fig_title='XYZ')

OSError: /home/adrian/RemoteFS/Rivendel/20200924/DC07_RF10/Try14/xvaSimuType4_01_N01024_Vrf0064_Udc0.4000D+01V_D1.0_S1.0RFG100eV.info not found.

In [260]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(mean(dim_nu[g,:,0])/mean(dim_nu[g,:,2]))
    y.append(mean(dim_nu[g,:,2])*1000)
    z.append(detect_rate[g])
    
    
fname = '2D_GMol_var_fluo_contour_Lvs_RoverL'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=250,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.set_xlabel(r'mean R/L')
ax.set_ylabel(r'mean L [µm]')

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=10,
                ha='center',va='center',weight='bold',zorder = 1000)
    # ax.annotate(str(txt), (x[i], y[i]-0.70),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.05))
ax.minorticks_on()
# ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.grid(which='major', linestyle='-', linewidth='1', color='k')
ax.grid(b=True, which='minor', color='k', linestyle=':')
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

In [261]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(mean(dim_nu[g,:,0])*1000)
    y.append(mean(dim_nu[g,:,2])*1000)
    z.append(detect_rate[g])
    
    
fname = '2D_GMol_var_fluo_contour_RvsL'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=250,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.set_xlabel(r'mean R [µm]')
ax.set_ylabel(r'mean L [µm]')

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=10,
                ha='center',va='center',weight='bold',zorder = 1000)
    # ax.annotate(str(txt), (x[i], y[i]-0.70),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.01*1000))
ax.minorticks_on()
# ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.grid(which='major', linestyle='-', linewidth='1', color='k')
ax.grid(b=True, which='minor', color='k', linestyle=':')
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

# SNR détaillé

In [123]:
plt.close('all')

In [79]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]

# my_plot = 1
# for my_plot,ll in enumerate(list(dict.fromkeys(points_and_coord))):

ll = 'test'
my_plot = 26

print(my_plot,ll)
my_base = 0.1

fname = 'SNR_'+'#'+str(my_plot)+'_'+ll
fig = figure(fname); clf()
fig.suptitle(r'SNR '+ll,color='xkcd:black',fontsize = 18)
ax1 = subplot(111)
my_bins = arange(4.65,5.25,0.005)
ax1.hist(Gmol_data[2][my_plot],bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('SNR')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)
savefig(fname+'.png') 

26 test


# Energie GMol

In [61]:
renorm = 0.5*m_GM/C_e

In [60]:
# Plot all fluo for all conditions
fname = 'DeltaEc_allpoints'
cm = pylab.get_cmap('rainbow')

fig, ax = plt.subplots(4, 7,num=fname,clear=True,sharex=True,sharey=True)  # (a,b)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

my_order = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,24,25,26,27,20,21,22,23]

my_base = 40
to_plot = 0
for l in range(0,7): # b
    for k in range(0,4): # a
        plot_point = my_order[to_plot]
#             str_eta_chiffre    = float(f'{eta[patate]:1.2e}'[:4])/10
#             str_eta_puissance  = int(re.sub('e','D',f'{eta[patate]:1.2e}'[6:]))-1
#             str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
        str_eta = '0.15D-19'

        my_bins = arange(-60,10,5)
        ax[3-k,l].hist(Gmol_data[0][plot_point]*renorm*1e3,bins=my_bins ,color='C5', edgecolor='black', linewidth=1.2)            
        ax[3-k,l].annotate(str(list(dict.fromkeys(points_and_coord))[plot_point]), xy=(0.02, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
        size=12, ha='left', va='top',
        bbox=dict(boxstyle='round', fc=cm(detect_rate[to_plot])))
        ax[3-k,l].grid()
        to_plot += 1
        if to_plot == 28:
            break
    if to_plot == 28:
        break
ax[3-k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax[3-k,l].yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=5))
# ax[3].set_xlim(-10,4000)
# ax[3-k,l].set_xlim(7.24,7.80)
ax[3-k,l].set_ylim(0,21)
# fig.suptitle(str(eta) + 'Kg.s-1')
fig.text(0.51, 0.04, r'$\Delta E_c$ [meV]', ha='center')
# fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.05, top = 0.98) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)

In [343]:
address = all_subdir[8*20+1]
print(address)
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
energy_lost(address,'xva'+sort(onlyfiles)[1][4:].strip('.dat'))[2] * renorm * 1000

/home/adrian/RemoteFS/Rivendel/20201006/DC05_RF08/Try01


-8.965338469105056

# Trajectory GMol

In [71]:
# Choose a specific point to consider
# Choose one point on the plot
plot_point = 4
condition_to_respect = list(dict.fromkeys(points_and_coord))[plot_point]
address_to_plot = [l for k,l in enumerate(all_subdir) if condition_to_respect in l]
for j,k in enumerate(address_to_plot):
    temp_files = listdir(k)
# Recover all GMol trajectories addresses
trajectory_files  = [[m for l,m in enumerate(temp_files) if 'trj' in m] for j,k in enumerate(address_to_plot)]
temperature_files = [[m for l,m in enumerate(temp_files) if 'Temp_SimuType4' in m] for j,k in enumerate(address_to_plot)]

# Recover all GMol trajectories addresses
trajectory_files  = [[m for l,m in enumerate(temp_files) if 'trj' in m] for j,k in enumerate(address_to_plot)]
temperature_files = [[m for l,m in enumerate(temp_files) if 'Temp_SimuType4' in m] for j,k in enumerate(address_to_plot)]
    
#trajectory_file = [listdir(j) for i,j in enumerate(address_to_plot)]
#trajectory_file = [k for j,k in enumerate(trajectory_file) if 'trj' in k]
print(trajectory_file)

temp_size = len(load_trj(os.path.join(address_to_plot[0],trajectory_files[0][0]).strip('.bin'))[0])
t, r_x,r_y,r_z, v_x,v_y,v_z, a_x,a_y,a_z = np.zeros((10,len(trajectory_files),temp_size))
tt, PM      = np.zeros((2,len(temperature_files),temp_size))
T_CM, T_aux = np.zeros((2,len(temperature_files),temp_size,3))
for j,k in enumerate(trajectory_files): # try
    print('>',j,'-',k)
    t[j], r_x[j],r_y[j],r_z[j], v_x[j],v_y[j],v_z[j], a_x[j],a_y[j],a_z[j] = load_trj(os.path.join(address_to_plot[j],k[0].strip('.bin')))
for j,k in enumerate(temperature_files): # try
    print('>',j,'-',k)
    tt[j], T_CM[j], T_aux[j], PM[j] = load_T_and_PM_simu(os.path.join(address_to_plot[j],k[0].strip('.dat')))

SNR_trig_threshold = 1

fname = 'plot_one_trj_'+condition_to_respect+'_'+str(E0_GMol)+'eV'
fig, ax = plt.subplots(2,1,num=fname,clear=True)

for k in range(len(r_z)):
    ax[0].plot(r_z[k]*1000,r_x[k]*1e9)
    ax[1].plot(r_z[k]*1000,r_y[k]*1e9)
    ax[1].set_xlabel('z [mm]')
    ax[0].set_ylabel('x [nm]')
    ax[1].set_ylabel('y [nm]')
    ax[0].set_ylim(-520,520)
    ax[1].set_ylim(-520,520)
    ax[1].grid()
    ax[0].grid()
fig.set_size_inches(11.69, 8.27)
tight_layout()

fname = 'plot_one_spexy_'+condition_to_respect+'_'+str(E0_GMol)+'eV'
fig, ax = plt.subplots(2,1,num=fname,clear=True)

for k in range(len(r_z)):
    ax[0].plot(r_z[k]*1000,v_x[k])
    ax[1].plot(r_z[k]*1000,v_y[k])
    ax[1].set_xlabel('z [mm]')
    ax[0].set_ylabel('v_x [m/s]')
    ax[1].set_ylabel('v_y [m/s]')
#     ax[0].set_ylim(-520,520)
#     ax[1].set_ylim(-520,520)
    ax[1].grid()
    ax[0].grid()
fig.set_size_inches(11.69, 8.27)
tight_layout()

fname = 'plot_one_spez_'+condition_to_respect+'_'+str(E0_GMol)+'eV'
fig, ax = plt.subplots(1,1,num=fname,clear=True)

for k in range(len(r_z)):
    ax.plot(r_z[k]*1000,v_z[k])
    ax.set_xlabel('z [mm]')
    ax.set_ylabel('v_z [m/s]')
#     ax[0].set_ylim(-520,520)
#     ax[1].set_ylim(-520,520)
    ax.grid()
fig.set_size_inches(11.69, 8.27)
tight_layout()

['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG6eV.bin', 'trj_SimuType4_01_N

In [36]:
# plot position r=sqrt(x**2+y**2) evolution of GMol
# during fly time
# depending space z
cm = pylab.get_cmap('gist_rainbow')   # colormap par défaut color=cm(k/len(array))

fname = 'GMol_trj_pos_allsublots'
fig, ax = plt.subplots(2, 2,num=fname,clear=True,sharex=True,sharey=True)
#title(all_subdir[0][:-15],fontsize=12)


for i in range(13-1):
    j = i//3 %2 #i%7
    k = i//6 %2
    w = t[i]
    x = r_z[i]
    y = sqrt(r_x[i]**2 + r_y[i]**2)
    cloud_size = dim_nu[points_to_plot,i,2]
    the_marker = trig_marker
    im = ax[j,k].scatter(x[22:40]*1000000,y[22:40]*1e9,
        marker=the_marker,s=55,lw=1.8,color='',edgecolor=cm(i/13))
    im = ax[j,k].scatter(x[22:40]*1000000,y[22:40]*1e9,
        marker='+',s=55,lw=1,color=cm(i/13))
    
i = 12    
j = 1 #i%7
k = 1
w = t[i]
x = r_z[i]
y = sqrt(r_x[i]**2 + r_y[i]**2)
cloud_size = dim_nu[points_to_plot,i,2]
if abs(point_SNR[i]) > SNR_trig_threshold:
    the_marker = trig_marker
    im = ax[j,k].scatter(x[22:40]*1000000,y[22:40]*1e9,
        marker=the_marker,s=55,lw=1.8,color='',edgecolor=cm(i/13))
    im = ax[j,k].scatter(x[22:40]*1000000,y[22:40]*1e9,
        marker='+',s=55,lw=1,color=cm(i/13))
else:
    the_marker = nontrig_marker
    im = ax[j,k].scatter(x[22:40]*1000000,y[22:40]*1e9,
        marker=the_marker,s=55,lw=3,color=cm(i/13))
ax[j,k].vlines(-cloud_size*1000,-10,75,color='xkcd:charcoal',linestyle=':',lw=2,zorder=0)
ax[j,k].vlines(cloud_size*1000,-10,75,color='xkcd:charcoal',linestyle=':',lw=2,zorder=0)  
#plt.setp(ax[j,k].get_xticklabels(), visible=False)
ax[j,k].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=200))
#ax[j,k].yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.02))

ax[1,0].set_xlabel(r'$r_z$ [µm]')
ax[1,1].set_xlabel(r'$r_z$ [µm]')
ax[0,0].set_ylabel(r'$\sqrt{r_x^2 + r_y^2}$ [nm]')
ax[1,0].set_ylabel(r'$\sqrt{r_x^2 + r_y^2}$ [nm]')
fig.suptitle('Distance radiale de la GMol par rapport à l\'axe de symétrie du piège')
#plt.setp(ax[6,0].get_xticklabels(), visible=True)
#plt.setp(ax[5,1].get_xticklabels(), visible=True,zorder=10000)

fig.set_size_inches(14, 10.5)

fig.tight_layout()
subplots_adjust(hspace=0.013,wspace=0.015,top=0.93)
savefig(fname+'_D'+str(Detuning)+'_r_'+str(whatisthisfile)+'.png',dpi=300)

NameError: name 'trig_marker' is not defined

# Distribution des vitesses

In [48]:
address = all_subdir[22]
# dirname_hc = str(data0[1][0])+'/'
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and 'SimuType' in f and '.dat' in f]
my_file = '{}/xva{}'.format(address,sort(onlyfiles)[2].strip('.dat')[4:])
print(my_file)
r_LC,v_LC,a_LC,v_LC_avg = load_xyz_init_bin_DP(my_file)  

/home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try22/xva_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG


In [49]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]
print(address)
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and "xva" in f and 'SimuTypeQ' in f and 'bin' in f]
r_LC, v_LC, a_LC, v_rf_avg = load_xyz_init_bin_DP(address+'/'+onlyfiles[0].strip('.bin'))


v_LC_x = v_LC[0]
v_LC_y = v_LC[1]
v_LC_z = v_LC[2]

my_base = 50

fig = figure('Terminal speed'); clf()
fig.suptitle(r'Atomic speed at the end of SimuType0 - RF - '+str(address[31:]),color='xkcd:black',fontsize = 18)
ax1 = subplot(311)
my_bins = arange(-250,251,20)
ax1.hist(v_LC_x,bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('$v_x$ [m/s]')

ax2 = subplot(312)
ax2.hist(v_LC_y,bins=my_bins ,color='C1', edgecolor='black', linewidth=1.2)
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax2.grid()
xlabel('$v_y$ [m/s]')

my_base = 0.2
ax3 = subplot(313)
my_bins = arange(-2,2.1,0.1)
# my_bins = arange(-30000,30001,5000)
ax3.hist(v_LC_z,bins=my_bins ,color='C2', edgecolor='black', linewidth=1.2)
ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax3.grid()
xlabel('$v_z$ [m/s]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

/home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try22


In [107]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]PleasePl
print(address)
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and "xva" in f and 'SimuTypeQ' in f and 'bin' in f]
r_LC, v_LC, a_LC, v_rf_avg = load_xyz_init_bin_DP(address+'/'+onlyfiles[0].strip('.bin'))
print(address+'/'+onlyfiles[0])


v_LC_avg_x = v_rf_avg[0]/100
v_LC_avg_y = v_rf_avg[1]/100
v_LC_avg_z = v_rf_avg[2]/100

my_base = 5

fig = figure('Terminal speed 2'); clf()
fig.suptitle(r'Atomic speed at the end of SimuType0 - no RF - '+str(address[31:]),color='xkcd:black',fontsize = 18)
ax1 = subplot(311)
my_bins = arange(-25,26,2)
ax1.hist(v_LC_avg_x,bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('$v_x$ [m/s]')

ax2 = subplot(312)
ax2.hist(v_LC_avg_y,bins=my_bins ,color='C1', edgecolor='black', linewidth=1.2)
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax2.grid()
xlabel('$v_y$ [m/s]')

my_base = 0.2
ax3 = subplot(313)
my_bins = arange(-2,2.1,0.1)
# my_bins = arange(-30000,30001,5000)
ax3.hist(v_LC_avg_z,bins=my_bins ,color='C2', edgecolor='black', linewidth=1.2)
ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax3.grid()
xlabel('$v_z$ [m/s]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

/home/adrian/RemoteFS/Hobitton/20200903/DC01_RF11/Try04
/home/adrian/RemoteFS/Hobitton/20200903/DC01_RF11/Try04/xva_SimuTypeQ_N01024_Vrf0070_Udc0.1000D+01V_D1.0_S1.0RFG.bin


In [93]:
m_kb_x_inv_n_ions  = m_Ca/(kb*N_ions*n_dt**2   )
T_aux_LC_x = sum(v_rf_avg[0]**2)*m_kb_x_inv_n_ions
T_aux_LC_y = sum(v_rf_avg[1]**2)*m_kb_x_inv_n_ions
T_aux_LC_z = sum(v_rf_avg[2]**2)*m_kb_x_inv_n_ions

fig = figure('Calculated temp'); clf()
fig.suptitle(r'Atomic speed at the end of SimuType0 - no RF - '+str(address[31:]),color='xkcd:black',fontsize = 18)

my_base = 1
my_bins = arange(0,1,0.005)

ax1 = subplot(311)
ax1.hist(T_aux_LC_x,bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('$v_x$ [m/s]')

ax2 = subplot(312)
ax2.hist(T_aux_LC_y,bins=my_bins ,color='C1', edgecolor='black', linewidth=1.2)
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax2.grid()
xlabel('$v_y$ [m/s]')

ax3 = subplot(313)
# my_bins = arange(-2,2.1,0.1)
# my_bins = arange(-30000,30001,5000)
ax3.hist(T_aux_LC_z,bins=my_bins ,color='C2', edgecolor='black', linewidth=1.2)
ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax3.grid()
xlabel('$v_z$ [m/s]')

fig.set_size_inches(11.69, 8.27)
# plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

In [132]:
n_threads = 32
space = N_ions/n_threads
linspace(1,1024-space+1,n_threads,dtype='int'),linspace(space,1024,n_threads,dtype='int')

(array([  1,  33,  65,  97, 129, 161, 193, 225, 257, 289, 321, 353, 385,
        417, 449, 481, 513, 545, 577, 609, 641, 673, 705, 737, 769, 801,
        833, 865, 897, 929, 961, 993]),
 array([  32,   64,   96,  128,  160,  192,  224,  256,  288,  320,  352,
         384,  416,  448,  480,  512,  544,  576,  608,  640,  672,  704,
         736,  768,  800,  832,  864,  896,  928,  960,  992, 1024]))

# Ion position

In [193]:
my_point = 20
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and 'SimuType' in f and '.dat' in f]
my_file = '{}/xva{}'.format(address,sort(onlyfiles)[2].strip('.dat')[4:])
r_LC,_,_,_ = load_xyz_init_bin_DP(my_file)
r_LC = np.zeros((len(num_runs),3,len(r_LC[0])))
for k,address in enumerate(all_subdir[my_point*len(num_runs):(my_point+1)*len(num_runs)]): # my_point*len(num_runs),(my_point+1)*len(num_runs)):
#     print(address)
    onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and 'SimuType' in f and '.dat' in f]
    my_file = '{}/xva{}'.format(address,sort(onlyfiles)[2].strip('.dat')[4:])
    print(my_file)
    r_LC[k],_,_,_ = load_xyz_init_bin_DP(my_file)

/home/adrian/RemoteFS/Rivendel/20200903/DC11_RF08/Try00/xva_SimuTypeQ_N01024_Vrf0053_Udc0.7000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC11_RF08/Try01/xva_SimuTypeQ_N01024_Vrf0053_Udc0.7000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC11_RF08/Try02/xva_SimuTypeQ_N01024_Vrf0053_Udc0.7000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC11_RF08/Try03/xva_SimuTypeQ_N01024_Vrf0053_Udc0.7000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC11_RF08/Try04/xva_SimuTypeQ_N01024_Vrf0053_Udc0.7000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC11_RF08/Try05/xva_SimuTypeQ_N01024_Vrf0053_Udc0.7000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC11_RF08/Try06/xva_SimuTypeQ_N01024_Vrf0053_Udc0.7000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC11_RF08/Try07/xva_SimuTypeQ_N01024_Vrf0053_Udc0.7000D+01V_D1.0_S1.0RFG
/home/adrian/RemoteFS/Rivendel/20200903/DC11_RF08/Try08/xva_SimuTypeQ_N01024_Vrf0053_Udc0.7000D+

In [113]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]PleasePl
my_base = 100

fig = figure('Ion pos at end cooling'); clf()
fig.suptitle(r'Ion pos at end cooling - '+str(address[31:]),color='xkcd:black',fontsize = 18)


ion = linspace(0,1023,1024,dtype='int')

for k in range(3):
    ax1 = subplot(311)
    my_bins = arange(-25,26,2)
    ax1.scatter(r_LC[k,2,ion]*1e3,r_LC[k,0,ion]*1e3, edgecolor='black', linewidth=1.2) # color='C0',
    ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
    ax1.axis('equal')
    ax1.grid()
    xlabel('$z$ [µm]')
    ylabel('$x$ [µm]')

    ax2 = subplot(312)
    ax2.scatter(r_LC[k,2,ion]*1e3,r_LC[k,1,ion]*1e3, edgecolor='black', linewidth=1.2)
    ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
    ax2.axis('equal')
    ax2.grid()
    xlabel('$z$ [µm]')
    ylabel('$y$ [µm]')

    ax3 = subplot(313)
    ax3.scatter(r_LC[k,0,ion]*1e3,r_LC[k,1,ion]*1e3, edgecolor='black', linewidth=1.2)
    ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
    ax3.set_aspect('equal', 'box')
    ax3.grid()
    xlabel('$x$ [µm]')
    ylabel('$y$ [µm]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [163]:
ion = linspace(0,1023,1024,dtype='int')
k = 0

fig = figure('Ion pos at end cooling - slicing - 3D'); clf()
ax1 = fig.add_subplot(111, projection='3d')

lower_x = -0.3
upper_x = +0.3

for k in range(25):
    x_slice = [x_pos*1e3 for n_atom,x_pos in enumerate(r_LC[k,0,:]) if lower_x<x_pos*1000<upper_x]
    y_slice = [y_pos*1e3 for n_atom,y_pos in enumerate(r_LC[k,1,:]) if lower_x<r_LC[k,0,n_atom]*1e3<upper_x]
    z_slice = [z_pos*1e3 for n_atom,z_pos in enumerate(r_LC[k,2,:]) if lower_x<r_LC[k,0,n_atom]*1e3<upper_x]
    ax1.scatter(x_slice,y_slice,z_slice, s=120, edgecolor='black', linewidth=2)
#     ax1.scatter(r_LC[k,0,ion]*1e3,r_LC[k,1,ion]*1e3,r_LC[k,2,ion]*1e3, edgecolor='black', linewidth=1.2)
#     ax1.axis('equal')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_zlabel('z')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()

In [194]:
ion = linspace(0,1023,1024,dtype='int')
k = 0

fig = figure('Ion pos at end cooling - 3D'); clf()
ax1 = fig.add_subplot(111, projection='3d')


for k in range(3):
    X = r_LC[k,0,ion]*1e3
    Y = r_LC[k,1,ion]*1e3
    Z = r_LC[k,2,ion]*1e3
    ax1.scatter(Z,Y,X, s=20, edgecolor='black', linewidth=1.2)
#     ax1.axis('equal')
#     ax1.set_xlabel('z')
#     ax1.set_ylabel('y')
#     ax1.set_zlabel('x')
    
ax1.set_xlim([-200,200])
ax1.set_ylim([-200,200])
ax1.set_zlim([-200,200])

plt.grid(b=None)
plt.axis('off')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()

# Miscellaneous

In [182]:
detect_cnt,len(all_subdir)

(213, 392)

In [19]:
# Pour toutes les simus confondues
# Pour chaque énergie incidente
i_Ec = [10, 50, 100, 1000]
d_rate = multiply([283/479,1562/2800,411/840,97/588],100)

fig = figure('detection_rate_vs_incident_energy'); clf()
ax1 = fig.add_subplot(111)


for k in range(3):
    ax1.semilogx(i_Ec,d_rate, marker='D', ms=15, color='C1', markeredgecolor='k', linestyle='')
ax1.set_xlabel('Incident energy [eV]')
ax1.set_ylabel('Detection rate [%]')

matplotlib.pyplot.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))
fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
savefig('detection_rate_vs_incident_energy',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

In [316]:
shape(Gmol_data[2]),shape(Gmol_data[2])[1]

((28, 30), 30)

In [264]:
# delta Ec moy pour toutes les simus
print(mean(Gmol_data[0])*renorm)
print(std(Gmol_data[0])*renorm)

-0.01449170079214501
0.012368744803265887


In [266]:
# delta Ec moy quand SNR > 1
# Sur tous les points confondus
detec_delta_Ec = []
for i in range(shape(Gmol_data[2])[0]):
    for j in range(shape(Gmol_data[2])[1]):
        if Gmol_data[2][i][j] > 1:
            detec_delta_Ec.append(Gmol_data[0][i][j])
# print(detec_delta_Ec)
print(len(detec_delta_Ec))
print(mean(detec_delta_Ec)*renorm)
print(std(detec_delta_Ec)*renorm)

411
-0.021845096260764835
0.013299722269260348


In [330]:
outfile = 'detection_rate_'+str(E0_GMol)+'eV'
# savefig(fname+'_D'+str(Detuning)+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)
np.savez(outfile, E0_GMol=E0_GMol, detect_rate=detect_rate, mean_std_deltaEc=mean_std_deltaEc, points_and_coord=points_and_coord, Udc=Udc,Urf=Urf)

In [331]:
with load(outfile+'.npz') as data:
    a = data['mean_std_deltaEc']
print(a)

[[-0.01388372  0.00084046 -0.01400457  0.00054802]
 [-0.01674034  0.0008823  -0.01674034  0.0008823 ]
 [-0.02037845  0.00106123 -0.02037845  0.00106123]
 [-0.02314676  0.00153862 -0.02314676  0.00153862]
 [-0.01409619  0.00117196         nan         nan]
 [-0.01800354  0.00059952 -0.01800354  0.00059952]
 [-0.01731739  0.00076923 -0.01731739  0.00076923]
 [-0.01612132  0.00109868 -0.01612132  0.00109868]
 [-0.00849097  0.00085012         nan         nan]
 [-0.01044078  0.00086746 -0.01088472  0.0004424 ]
 [-0.01358862  0.00116915 -0.01358862  0.00116915]
 [-0.01679448  0.00077754 -0.01679448  0.00077754]
 [-0.00989987  0.00063993         nan         nan]
 [-0.00879481  0.00097469 -0.00825804  0.        ]
 [-0.00943289  0.00072419 -0.00965308  0.0006013 ]
 [-0.01126605  0.00075449 -0.01126605  0.00075449]
 [-0.00742889  0.00100172         nan         nan]
 [-0.00983632  0.00081443         nan         nan]
 [-0.00911523  0.00103437 -0.00945305  0.00086701]
 [-0.00918776  0.00045762 -0.00